### > Time took to create a DF of all (4, 5) k-mers sequences with occurences:  7953.4810598999975 seconds

### > This was very slow with kmer 6, more than 9 hours ongoing, and I had to kill the process to liberate resources.



In [103]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
# import swifter
import gc
import timeit
# from numba import jit, cuda
import dask.dataframe as dd
from dask.multiprocessing import get
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, plot_confusion_matrix
from scipy import sparse
import seaborn as sn
from joblib import dump, load
import multiprocessing as mp
from pandarallel import pandarallel

%matplotlib inline

In [104]:
labels = []
def importLabels():
#     with open('./Data/shortlabels.txt', 'r') as filehandle:
    with open('./Data/labels.txt', 'r') as filehandle:


        for line in filehandle:
            # remove linebreak, if exists, which is the last character of the string
            if(line[-1] == "\n"):
                currentPlace = line[:-1]
            else:
                currentPlace = line[:]
            # add item to the list
            labels.append(currentPlace)
importLabels()

In [105]:
identifiers = [] # list
sequences = [] # list

with open('./Data/sequences.fasta') as fasta_file:  # Will close handle cleanly
# with open('./Data/short.fasta') as fasta_file:  # Will close handle cleanly

    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequences.append(str(seq_record.seq))
print(len(sequences))

60000


In [106]:
# kmrdic = pd.DataFrame()


def kmersPerSequenceDict(sequence, lower, upper):
    dic =  {}
    for size in range(lower, upper+1):
        for x in range(len(sequence) - size + 1):
            if not str(sequence[x:x+size].lower()) in dic:
                dic[str(sequence[x:x+size].lower())] = 1
            else:
                dic[str(sequence[x:x+size].lower())] += 1
    return dic
    print('type of dic :', type(dic))

# time required to process SINGLE sequence for kmers between 5-6
### Time took to create concatenated k-mers sequences:  0.04519199999776902

In [107]:
# t_pds = timeit.Timer(lambda: genKmersDictionary(sequences[0], 5, 6))
# print("Time took to create concatenated k-mers sequences: ",  t_pds.timeit(number=1))


In [108]:
output = pd.DataFrame()

In [109]:
# df = pd.concatenate([genKmers(sequences[i], 5, 6) for i in range(len(sequences))])

lower = 5
upper = 6
def kmersPerDataset():
    global output
    for sequence in sequences:
        dic = kmersPerSequenceDict(sequence, lower, upper)
        output = output.append(dic, ignore_index=True)

In [ ]:
t_pds = timeit.Timer(lambda: kmersPerDataset())
print("Time took to create a DF of all (" + str(lower) + "," + str(upper)+") k-mers sequences with occurences: " +  str(t_pds.timeit(number=1)) + "seconds")

In [ ]:
display(output)

In [ ]:
output.to_pickle('./saved/5-6-Kmers-60,000 samples') 